In [5]:
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import os
import natsort
from torch.utils import data
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import random
from tqdm.auto import tqdm
import pytorch_lightning
seed = 12345
random.seed(seed)
torch.manual_seed(seed)

In [14]:
train = pd.read_csv('ranzcr-clip-catheter-line-classification/train.csv')
train_annotations = pd.read_csv('ranzcr-clip-catheter-line-classification/train_annotations.csv')
tet = 

In [7]:
train.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [8]:
train['ETT - Abnormal'].sum()

79

In [9]:
train['ETT - Borderline'].sum()

1138

In [10]:
train['ETT - Normal'].sum()

7240

In [60]:
test_names = pd.read_csv('/Users/williamegan/Desktop/ranzcr/ranzcr-clip-catheter-line-classification/test.txt', header = None)

In [83]:
test_ids = pd.DataFrame(test_names[0].str.slice(start = 5, stop = -4)).rename(columns = {0:'ids'}).copy()
test_ids

,ids
0,1.2.826.0.1.3680043.8.498.10003659706701445041...
1,1.2.826.0.1.3680043.8.498.10003890246067211044...
2,1.2.826.0.1.3680043.8.498.10008058926714428108...
3,1.2.826.0.1.3680043.8.498.10008122048552553565...
4,1.2.826.0.1.3680043.8.498.10010309624955962953...
...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...


In [84]:
#create fake labels. we do this just so we can use the CustomDataset class
test = pd.concat([test_ids, pd.DataFrame(np.zeros((len(test_ids),11)))], axis=1)
test

,ids,0,1,2,3,4,5,6,7,8,9,10
0,1.2.826.0.1.3680043.8.498.10003659706701445041...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.2.826.0.1.3680043.8.498.10003890246067211044...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.2.826.0.1.3680043.8.498.10008058926714428108...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.2.826.0.1.3680043.8.498.10008122048552553565...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.2.826.0.1.3680043.8.498.10010309624955962953...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.99840913070180372408...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3578,1.2.826.0.1.3680043.8.498.99893249190615412121...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3579,1.2.826.0.1.3680043.8.498.99930578228271951378...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3580,1.2.826.0.1.3680043.8.498.99953100661957840394...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# labels_dict = {train.iloc[i][0]:torch.Tensor(train.iloc[0][1:-1]) 
#               for i in range(len(train))}

In [72]:
labels_dict = {}
for i in range(len(train)):
    k = train.iloc[i][0]
    v = torch.Tensor(train.iloc[0][1:-1])
    labels_dict[k] = v

In [86]:
fake_test_labels = {}
for i in range(len(test)):
    k = test.iloc[i]['ids']
    v = torch.Tensor(test.iloc[0][1:-1])
    fake_test_labels[k] = v

In [87]:
fake_test_labels

{'1.2.826.0.1.3680043.8.498.10003659706701445041816900371598078663': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10003890246067211044742686138544513464': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10008058926714428108535101002094531530': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10008122048552553565774139897939785611': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10010309624955962953138679469906703941': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10016971606732284006071899723342963784': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10020268825604182610368224629707426511': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10023042737818625910026668901358652653': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10023687805200508996

In [70]:
labels_dict

{'1.2.826.0.1.3680043.8.498.26697628953273228189375557799582420561': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.46302891597398758759818628675365157729': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.23819260719748494858948050424870692577': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.68286643202323212801283518367144358744': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.10050203009225938259119000528814762175': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.11707076266253086830850990314496191758': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.23133310896792275593685843424784127411': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.3680043.8.498.17332833449968516367349904809269946843': tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 '1.2.826.0.1.36

In [25]:
#the commented things here can change the images to become 3 channel
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labels):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)
        self.labels = labels

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)#.convert("RGB")
        tensor_image = self.transform(image)
        key = os.path.basename(img_loc)[:-4]#this is how we get our ids
        #input labels as dictionary with id number as the key
        label = self.labels[key]
        return tensor_image, label

train_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])


test_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])

In [92]:
images = CustomDataSet('/Users/williamegan/Desktop/ranzcr/ranzcr-clip-catheter-line-classification/train',train_transform, labels_dict)
images_test = CustomDataSet('/Users/williamegan/Desktop/ranzcr/ranzcr-clip-catheter-line-classification/test',test_transform, fake_test_labels)

In [24]:
images[0]

(tensor([[[-2.2338, -2.2521, -2.2521,  ..., -2.2521, -2.2521, -2.2521],
          [-2.0141, -2.0324, -2.0690,  ..., -2.2521, -2.2521, -2.2521],
          [-1.9775, -1.9775, -1.9775,  ..., -2.2521, -2.2521, -2.2521],
          ...,
          [ 0.1829,  0.4758,  0.6223,  ..., -1.2818, -1.5381, -2.1239],
          [ 0.2378,  0.5307,  0.6772,  ..., -1.3184, -1.5747, -2.1239],
          [ 0.2378,  0.5307,  0.6589,  ..., -1.4099, -1.6846, -2.1423]]]),
 tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]))

In [14]:
len(images)

30083

In [94]:
images_test[0]

(tensor([[[-2.1239, -2.1239, -2.1239,  ..., -2.1239, -2.1239, -2.1239],
          [-2.1239, -2.1239, -2.1239,  ..., -2.1239, -2.1423, -2.1423],
          [-2.1423, -2.1423, -2.1239,  ..., -2.0324, -1.8127, -1.7944],
          ...,
          [-2.1056, -2.1423, -2.1423,  ..., -1.5747, -2.0690, -2.1606],
          [-2.1423, -2.1423, -2.1423,  ..., -1.5564, -2.0690, -2.1606],
          [-2.1606, -2.1423, -2.1423,  ..., -1.5564, -2.0690, -2.1606]]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [97]:
#this is the splitting routine for the train/val split
random_seed = 12345
validation_split = .2
shuffle_dataset = True

dataset_size = len(images)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


In [98]:
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

In [99]:
batch_size = 256
train_loader = torch.utils.data.DataLoader(images, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers = 4)
val_loader = torch.utils.data.DataLoader(images, batch_size=batch_size,
                                                sampler=val_sampler, num_workers = 4)


test_loader = torch.utils.data.DataLoader(images_test, batch_size=batch_size,
                                                num_workers = 4)


In [101]:
test_loader[

TypeError: 'DataLoader' object is not subscriptable

## Sandbox Area

In [102]:
for b in test_loader:
    batch = b
    break

In [25]:
for b in val_loader:
    batch = b
    break

In [103]:
data, target = batch

In [106]:
data.size()

torch.Size([256, 1, 48, 48])

In [66]:
model = ShallowConvnet(1, 11)

In [67]:
ypred = model(data)

In [69]:
ypred

tensor([[0.4699, 0.5347, 0.5091,  ..., 0.4209, 0.6092, 0.5359],
        [0.4714, 0.5354, 0.5123,  ..., 0.4095, 0.6224, 0.5437],
        [0.4690, 0.5273, 0.5045,  ..., 0.4204, 0.6061, 0.5339],
        ...,
        [0.4854, 0.5333, 0.5009,  ..., 0.4237, 0.6019, 0.5446],
        [0.4730, 0.5478, 0.5085,  ..., 0.4187, 0.6088, 0.5163],
        [0.4773, 0.5431, 0.4974,  ..., 0.4086, 0.6307, 0.5389]],
       grad_fn=<SigmoidBackward>)

In [70]:
#next step is to convert this to 0's and ones
ypred.round()

tensor([[0., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        ...,
        [0., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 0., 1., 1.]], grad_fn=<RoundBackward>)

In [71]:
each_ex_acc = (target == ypred.round()).sum(dim=1)/len(target[0])# for each example in the batch

In [81]:
ypred.no_grad()

AttributeError: 'Tensor' object has no attribute 'no_grad'

In [77]:
target

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [83]:
loss = torch.nn.MultiLabelSoftMarginLoss()
loss(ypred, target)

tensor(0.9347, grad_fn=<MeanBackward0>)

In [72]:
each_ex_acc.sum()/len(target)

tensor(0.3928)

With the above, we have found the way we want to present our predictions. The softmax of logits give us probabilities just as the thingy dingus requires. 

Now, we must find a way to evaluate these logits for a multiclass AUROC

i guess what we are doing is predicting whether each column is a 0 or 1. 
Which means that for each of them, there are two options. For now, we will assume that these can be determined independently? I guess so, just to get this set up right. 

Okay so we want to see if 

WE JUST GOTTA HIT EM EACH WITH A SIGNMOOID

In [65]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)
    
    
    
class ShallowConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        """

        Parameters
        ----------
        input_channels : Number of input channels
        num_classes : Number of classes for the final prediction 
        """
        
        super().__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes
        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 8)
        )
#         self.conv1 = nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2)
#         self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1)
        self.fc = nn.Linear(256, self.num_classes)
        self.sig = nn.Sigmoid()


    def forward(self, x):
        
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)
            
        x = View((-1,256))(x)
            
        x = self.fc(x)
        x = self.sig(x)
        return x



        
    
class SimpleConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet, self).__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU()
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 8)
        )

        #final linear layer to project into the correct number of classes
        self.fc = nn.Linear(256, 11)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
       
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = View((-1,256))(x)
        x = self.fc(x)
        x = self.sig(x)
        output = x
        
        return output
    
    

In [58]:
def train_loop(model, criterion, optimizer,  train_loader, val_loader):
    """
    Generic training loop

    Parameters
    ----------
    model : Object instance of your model class 
    criterion : Loss function 
    optimizer : Instance of optimizer class of your choice 
    train_loader : Training data loader 
    val_loader : Validation data loader

    Returns
    -------
    train_losses : List with train loss on dataset per epoch
    train_accuracies : List with train accuracy on dataset per epoch
    val_losses : List with validation loss on dataset per epoch
    val_accuracies : List with validation accuracy on dataset per epoch

    """
    best_val = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    max_patience = 5
    patience_counter = 0
    # Training
    for t in tqdm(range(50)):
        epoch_t_acc = 0.0 
        epoch_t_loss = 0.0
        model.train()       
        for i, samples in enumerate(train_loader):
            data, target = samples
            y_pred_train = model(data)

            loss = criterion(y_pred_train, target)
#             score, predicted = torch.max(y_pred_train, 1)
#             acc = (predicted == target).sum().float() / len(target)
            # for each example in the batch
            each_ex_acc = (target == ypred.round()).sum(dim=1)/len(target[0])
            acc = each_ex_acc.sum()/len(target)
            
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_t_acc += acc
            epoch_t_loss += loss.item()

        train_accuracies.append(epoch_t_acc/len(train_loader))
        train_losses.append(epoch_t_loss/len(train_loader))

        model.eval()

        v_acc = 0.0
        v_loss = 0.0


        with torch.no_grad():
        # TODO: Loop over the validation set 
            for i, samples in enumerate(val_loader):

                # TODO: Put the inputs and targets on the write device
                data, target = samples
                #data, target = data.to(device), target.to(device)

                # TODO: Feed forward to get the logits
                y_pred_val = model(data)

                # TODO: Compute the loss and accuracy
                loss = criterion(y_pred_val, target)
#                 score, predicted = torch.max(y_pred_val, 1)
#                 acc = (predicted == target).sum().float() / len(target)
                each_ex_acc = (target == ypred.round()).sum(dim=1)/len(target[0])
                acc = each_ex_acc.sum()/len(target)
        
                v_loss+= loss.item()
                v_acc += acc


        # TODO: Keep track of accuracy and loss
        val_accuracies.append(v_acc/len(val_loader))
        val_losses.append(v_loss/len(val_loader))

        if val_accuracies[-1] > best_val:
            best_val = val_accuracies[-1]
            patience_counter = 0

      # TODO: Save best model, optimizer, epoch_number
            torch.save({
              'model': model.state_dict(),
              'optimizer': optimizer.state_dict(),
              'epoch': t,
            }, '/content/drive/My Drive/Colab Notebooks/checkpoint.pth')

        else:
            patience_counter += 1    
            if patience_counter > max_patience: 
                break

        print("[EPOCH]: %i, [TRAIN LOSS]: %.6f, [TRAIN ACCURACY]: %.3f" % (t, train_losses[-1], train_accuracies[-1]))
        print("[EPOCH]: %i, [VAL LOSS]: %.6f, [VAL ACCURACY]: %.3f \n" % (t, val_losses[-1] ,val_accuracies[-1]))

    return train_losses, train_accuracies, val_losses, val_accuracies

In [23]:
val_loader

NameError: name 'val_loader' is not defined

In [8]:
# TODO : Initialize the model and cast to correct device
input_channels = 1
num_classes = 11
model_sc = cf.SimpleConvnet(input_channels, num_classes)
#model_sc.to(cf.device)

# TODO : Initialize the criterion
criterion = torch.nn.CrossEntropyLoss()
# TODO : Initialize the SGD optimizer with lr 1e-3
optimizer = optim.SGD(model_sc.parameters(), lr = 0.001)

# TODO : Run the training loop using this model

train_losses, train_accuracies, val_losses, val_accuracies = train_loop(model_sc, criterion, optimizer, train_loader, val_loader)




NameError: name 'cf' is not defined